In [2]:
import pandas as pd
import datetime 
import numpy as np
import math

In [3]:
df=pd.read_csv('inputreport.csv')


In [4]:
my_list = df.columns.values.tolist()

In [5]:
df['Scheduled Visit End'] = df['Scheduled Visit End'].astype('datetime64[ns]')
df['Scheduled Visit Start'] = df['Scheduled Visit Start'].astype('datetime64[ns]')
df["Scheduled Visit Start"].dtype

dtype('<M8[ns]')

In [6]:
df['SVS'] = df['Scheduled Visit Start']
df['SVE'] = df['Scheduled Visit End']
df["SVS"].dtype

dtype('<M8[ns]')

In [7]:
df=df.sort_values(by=['Scheduled Visit Start','Participant First Name'])

In [8]:
df['DayOftheWeek'] = df['Scheduled Visit Start'].dt.dayofweek
df["WeekOfyear"] = df['Scheduled Visit Start'].dt.isocalendar().week
df["Day"] = df['Scheduled Visit Start'].dt.day
df['Month'] = df['Scheduled Visit Start'].dt.month
df['Year'] = df['Scheduled Visit Start'].dt.year


In [9]:
df['hourDigit'] = df['Scheduled Visit Start'].dt.hour
df['Actual Visit Start'] = df['Actual Visit Start'].astype('datetime64[ns]')
df['Actual Visit End'] = df['Actual Visit End'].astype('datetime64[ns]')
df['SchedMin']=df['Scheduled Visit End']-df['Scheduled Visit Start']
df["SchedMin"] = df["SchedMin"].to_numpy()/np.timedelta64(1, 'm')
df['AcctMIn']=df['Actual Visit End']-df['Actual Visit Start']
df['AcctMIn'] = df['AcctMIn'].to_numpy()/np.timedelta64(1, 'm')
df['StartDif']=df['Actual Visit Start']-df['Scheduled Visit Start']
df["StartDif"] = df["StartDif"].to_numpy()/np.timedelta64(1, 'm')
df['EndDif']=(df['Actual Visit End']-df['Scheduled Visit End'])
df["EndDif"] = df["EndDif"].to_numpy()/np.timedelta64(1, 'm')
df['HoursDif']=(df['AcctMIn']-df['SchedMin'])/60.00
df['Untis']=df['SchedMin']/15
df['Difference']=df['AcctMIn']-df['SchedMin']

In [10]:
df=df.sort_values(by=['Scheduled Visit Start'])
df['Code']=+df['hourDigit'].map(str)+"_"+df['Employee First Name'].map(str)+"_"+df['Participant First Name'].map(str)
df=df.sort_values(by=['Code','Scheduled Visit Start'])

In [11]:
df['Timelaps']=df["Scheduled Visit Start"].diff()
df['Timelaps'].diff().apply(lambda x: x/np.timedelta64(1, 'D')).fillna(0).astype('int64')
df['Timelaps'] = df['Timelaps'].to_numpy()/np.timedelta64(1, 'D')
def truncf(num):
    if num == None:
        num1=0
    else:
        num1=float(num)
    return num1
df['Timelaps'] = df['Timelaps'].apply(truncf).fillna(0)
def trunci(num):
    num1=int(num)
    return num1
df['Timelaps'] = df['Timelaps'].apply(trunci)
df.loc[df['Timelaps'] <=1,"Split"]=0
df.loc[df['Timelaps'] >1,"Split"]=1
df["Split"]=df["Split"].astype(int)
df['split_csum'] = df.groupby(['WeekOfyear'])['Split'].cumsum()

In [12]:
df['Count']=1

In [13]:
df['Grouping']=df["Month"].map(str)+"_"+df["WeekOfyear"].map(str)+"_"+df["split_csum"].map(str)

In [14]:
df['MaxSVS'] = df.groupby('Grouping')["Day"].transform('max')
df['MinSVS'] = df.groupby('Grouping')["Day"].transform('min')

In [15]:
df1=df

In [16]:
df['StartDate']=df["Month"].map(str)+"/"+df["MinSVS"].map(str)+"/"+df["Year"].map(str)
df['EndDate']=df["Month"].map(str)+"/"+df["MaxSVS"].map(str)+"/"+df["Year"].map(str)

In [17]:
g1=df1.groupby(['Code','Grouping','StartDate','EndDate'],as_index=False).sum()
g1.add_suffix('_Count').reset_index()

,index,Code_Count,Grouping_Count,StartDate_Count,EndDate_Count,Notes_Count,DayOftheWeek_Count,WeekOfyear_Count,Day_Count,Month_Count,...,EndDif_Count,HoursDif_Count,Untis_Count,Difference_Count,Timelaps_Count,Split_Count,split_csum_Count,Count_Count,MaxSVS_Count,MinSVS_Count
0,0,13_John_Smith,11_45_0,11/13/2021,11/14/2021,0.0,11,90,27,22,...,61.266667,1.010556,60.0,60.633333,1,0,0,2,28,26
1,1,13_John_Smith,12_48_1,12/4/2021,12/4/2021,0.0,5,48,4,12,...,0.533333,0.004167,32.0,0.250000,20,1,1,1,4,4
2,2,13_John_Smith,12_49_1,12/11/2021,12/12/2021,0.0,11,98,23,24,...,60.566667,0.996389,60.0,59.783333,8,1,2,2,24,22
3,3,13_John_Smith,12_50_1,12/18/2021,12/19/2021,0.0,11,100,37,24,...,510.966667,8.196111,60.0,491.766667,7,1,2,2,38,36
4,4,13_John_Smith,12_51_1,12/25/2021,12/25/2021,0.0,5,51,25,12,...,0.466667,-0.010833,32.0,-0.650000,6,1,1,1,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,103,8_John_Smith,4_13_4,4/2/2022,4/3/2022,0.0,11,26,5,8,...,1150.683333,19.130278,40.0,1147.816667,7,1,8,2,6,4
104,104,8_John_Smith,4_14_4,4/9/2022,4/10/2022,0.0,11,28,19,8,...,1.700000,0.021944,40.0,1.316667,7,1,8,2,20,18
105,105,8_John_Smith,4_15_4,4/16/2022,4/17/2022,0.0,11,30,33,8,...,2.450000,0.014722,40.0,0.883333,7,1,8,2,34,32
106,106,8_John_Smith,4_16_4,4/23/2022,4/24/2022,0.0,11,32,47,8,...,-18.216667,0.576389,40.0,34.583333,7,1,8,2,48,46


In [18]:
g2=g1[['Code','StartDate','EndDate','Untis']]
g2

,Code,StartDate,EndDate,Untis
0,13_John_Smith,11/13/2021,11/14/2021,60.0
1,13_John_Smith,12/4/2021,12/4/2021,32.0
2,13_John_Smith,12/11/2021,12/12/2021,60.0
3,13_John_Smith,12/18/2021,12/19/2021,60.0
4,13_John_Smith,12/25/2021,12/25/2021,32.0
...,...,...,...,...
103,8_John_Smith,4/2/2022,4/3/2022,40.0
104,8_John_Smith,4/9/2022,4/10/2022,40.0
105,8_John_Smith,4/16/2022,4/17/2022,40.0
106,8_John_Smith,4/23/2022,4/24/2022,40.0


In [19]:
g2.to_csv('ouput2.csv')

In [20]:
df.to_csv('output.csv')